In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size, 128'),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
    )
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):


  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [15]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters = 128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [16]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
1088/1088 [==============================] - 13s 12ms/step - loss: 1.5129 - accuracy: 0.5534 - val_loss: 0.3692 - val_accuracy: 0.8833
Epoch 2/5
1088/1088 [==============================] - 11s 10ms/step - loss: 0.3684 - accuracy: 0.8905 - val_loss: 0.1985 - val_accuracy: 0.9408
Epoch 3/5
1088/1088 [==============================] - 11s 10ms/step - loss: 0.2485 - accuracy: 0.9316 - val_loss: 0.0693 - val_accuracy: 0.9814
Epoch 4/5
1088/1088 [==============================] - 11s 10ms/step - loss: 0.2069 - accuracy: 0.9468 - val_loss: 0.0863 - val_accuracy: 0.9765
Epoch 5/5
1088/1088 [==============================] - 11s 10ms/step - loss: 0.2220 - accuracy: 0.9442 - val_loss: 0.0465 - val_accuracy: 0.9878


0.9684807256235828

In [11]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1716 - accuracy: 0.9576


0.9575963616371155

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters = 32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters = 32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters = 64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters = 128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters = 128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
   )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [14]:
space = {
    'batch_size': hp.quniform('batch_size', 100,200,10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3,0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3,0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3,0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3,0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3,0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.39219745262507355, 'dropout_cnn_block_three': 0.3419638471256517, 'dropout_cnn_block_two': 0.4636720026761296, 'dropout_dense_block_one': 0.6129903624009669, 'dropout_dense_block_two': 0.6983795064900751}
accuracy=0.7641723155975342
{'batch_size': 130.0, 'dropout_cnn_block_one': 0.421585813283002, 'dropout_cnn_block_three': 0.4006627787729139, 'dropout_cnn_block_two': 0.35947674068540675, 'dropout_dense_block_one': 0.5548708592178351, 'dropout_dense_block_two': 0.5918836009824993}
accuracy=0.9095237851142883
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.31174447026592433, 'dropout_cnn_block_three': 0.46023352093183534, 'dropout_cnn_block_two': 0.4165662912334577, 'dropout_dense_block_one': 0.32381055607769954, 'dropout_dense_block_two': 0.3403747699351334}
accuracy=0.9639455676078796
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3223960476560546, 'dropout_cnn_block_three': 0.4422065979990152, 'dropout_cnn_block_two': 0.399846875887

In [14]:
space = {
    'batch_size': hp.quniform('batch_size', 50,200,20),
    'dropout_cnn_block_one': 0.5,#hp.uniform('dropout_cnn_block_one', 0,1),
    'dropout_cnn_block_two': 0.5,#hp.uniform('dropout_cnn_block_two', 0,1),
    'dropout_cnn_block_three': 0.5,#hp.uniform('dropout_cnn_block_three', 0,1),

    'dropout_dense_block_one': 0.5,#hp.uniform('dropout_dense_block_one', 0,1),
    'dropout_dense_block_two': 0.5,#hp.uniform('dropout_dense_block_two', 0,1),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

SyntaxError: ignored

ERROR! Session/line number was not unique in database. History logging moved to new session 59
